In [1]:
#1.3 ellipsoid
# Let M be an ellipsoid in R3, (ax)^{2}+(by)^{2}+z^{2}=d
# x = 1/a*cos(p)*sin(t)*e1 + 1/b*sin(p)*sin(t)*e2 + cos(t)*e3, t, p in R

from gc_utils import *
alg = Algebra(3)
locals().update(alg.blades)

c2e = lambda p, t: 4*np.cos(p)*np.sin(t)*e1 + 5*np.sin(p)*np.sin(t)*e2 + np.cos(t)*e3
e_p = lambda p, t: -4*np.sin(p)*np.sin(t)*e1 + 5*np.cos(p)*np.sin(t)*e2
e_t = lambda p, t: (4*np.cos(p)*np.cos(t)*e1) + (5*np.sin(p)*np.cos(t)*e2) - (np.sin(t)*e3)
def Ix(x):
    p = np.arctan((4/5)*x.e2/x.e1)
    t = np.arccos(x.e3)
    return e_p(p, t) ^ e_t(p, t)

# |p| < pi/2, 0 < t < pi for unique arctan & arccos
p, t = -1, 3
x = c2e(p, t)
I = Ix(x)
nIx = lambda x: normalize(Ix(x))
nI = nIx(x)

nI, nI ^ (3*e_t(p,t) + 5*e_p(p,t))

(1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃, 1.29e-15 𝐞₁₂₃)

In [2]:
#5.1
C, D, E = [random_multivector(alg) for _ in range(3)]
V1, V2 = [random_versor(2, alg) for _ in range(2)]
f = lambda x: V1.sw(inv(x)) + V2.sw(x**3)
A = lambda x: E*x**2 + C*x*D
a = lambda x: A(x).grade(1)
PA = lambda x: P(A(x), Ix(x))
pa = lambda x: P(a(x), Ix(x))
f_ = lambda a: lambda x: differential(f, x, a)

x, f(x), pa(x), f_(pa(x))(x)

(0.305 𝐞₁ + -0.594 𝐞₂ + -0.99 𝐞₃,
 -1.42 𝐞₁ + 0.453 𝐞₂ + -4.25 𝐞₃,
 0.0808 𝐞₁ + -1.27 𝐞₂ + 0.0319 𝐞₃,
 -2.1 𝐞₁ + -3.5 𝐞₂ + -5.1 𝐞₃)

In [3]:
#5.2 For tangent fields, the outermorphism can be computed in the embedded space
def outermorphism(f, field, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        A = field(x)
        wf = lambda vectors: wedge([f(v)(x) for v in vectors])
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
        return outer
    return f_outer

outermorphism(f_, A, alg)(x), outermorphism(f_, A, alg, Ix=Ix)(x), outermorphism(f_, PA, alg)(x), outermorphism(f_, PA, alg, Ix=Ix)(x)

(0.258 + -1.73 𝐞₁ + -1.33 𝐞₂ + -10.9 𝐞₃ + 5.98 𝐞₁₂ + -19.0 𝐞₁₃ + -20.2 𝐞₂₃ + -76.6 𝐞₁₂₃,
 0.258 + -2.1 𝐞₁ + -3.5 𝐞₂ + -5.1 𝐞₃ + 0.0727 𝐞₁₂ + 0.114 𝐞₁₃ + 0.0125 𝐞₂₃ + -2.29e-17 𝐞₁₂₃,
 0.258 + -2.1 𝐞₁ + -3.5 𝐞₂ + -5.1 𝐞₃ + 0.0727 𝐞₁₂ + 0.114 𝐞₁₃ + 0.0125 𝐞₂₃,
 0.258 + -2.1 𝐞₁ + -3.5 𝐞₂ + -5.1 𝐞₃ + 0.0727 𝐞₁₂ + 0.114 𝐞₁₃ + 0.0125 𝐞₂₃ + -2.29e-17 𝐞₁₂₃)

In [4]:
#5.3
f_vec = lambda a: lambda x: differential(f, x, a)
f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
I1 = f_(Ix)(x)
nI1 = f_(nIx)(x)
norm(I1)/norm(I), norm(nI1)

(21.09431527887049, 21.094315278870496)

In [5]:
# Jacobian
Jf = lambda x: norm(f_(nIx)(x))

In [6]:
#5.4
def adjoint_outermorphism(f, field, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        A = field(x)
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        F = lambda vectors: wedge([f(v)(x) for v in vectors]).sp(A)
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            # Why np.zeros(r)? The derivative of a linear function is constant
            outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
        return outer
    return f_outer

def adjoint_outermorphism_(f, A, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        F = lambda vectors: wedge([f(v)(x) for v in vectors]).sp(A)
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            # Why np.zeros(r)? The derivative of a linear function is constant
            outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
        return outer
    return f_outer

adjoint_outermorphism(f_vec, A, alg, Ix=Ix)(x)

0.258 + 5.04 𝐞₁ + -6.11 𝐞₂ + 0.244 𝐞₃ + -6.75 𝐞₁₂ + 0.162 𝐞₁₃ + 0.13 𝐞₂₃

In [7]:
#5.10a
f_(A)(x), f_(PA)(x), P(f_(A)(x), I1)

(0.258 + -2.1 𝐞₁ + -3.5 𝐞₂ + -5.1 𝐞₃ + 0.0727 𝐞₁₂ + 0.114 𝐞₁₃ + 0.0125 𝐞₂₃ + -2.29e-17 𝐞₁₂₃,
 0.258 + -2.1 𝐞₁ + -3.5 𝐞₂ + -5.1 𝐞₃ + 0.0727 𝐞₁₂ + 0.114 𝐞₁₃ + 0.0125 𝐞₂₃ + -2.29e-17 𝐞₁₂₃,
 0.258 + -2.1 𝐞₁ + -3.5 𝐞₂ + -5.1 𝐞₃ + 0.0727 𝐞₁₂ + 0.114 𝐞₁₃ + 0.0125 𝐞₂₃)

In [8]:
#5.10b
P_ = lambda A: lambda x: P(A(x), I)
P__ = lambda A: lambda x: P(A(x), I1)
fa = lambda A: adjoint_outermorphism(f_vec, A, alg, Ix=Ix)
fa(A)(x), fa(P__(A))(x), P_(fa(A))(x)

(0.258 + 5.04 𝐞₁ + -6.11 𝐞₂ + 0.244 𝐞₃ + -6.75 𝐞₁₂ + 0.162 𝐞₁₃ + 0.13 𝐞₂₃,
 0.258 + 5.04 𝐞₁ + -6.11 𝐞₂ + 0.244 𝐞₃ + -6.75 𝐞₁₂ + 0.162 𝐞₁₃ + 0.13 𝐞₂₃,
 0.258 + 5.04 𝐞₁ + -6.11 𝐞₂ + 0.244 𝐞₃ + -6.75 𝐞₁₂ + 0.162 𝐞₁₃ + 0.13 𝐞₂₃)

In [9]:
#5.12
G = lambda x: (x+D)**2
F = lambda x: G(f(x))
differential(F, x, pa(x)), differential(G, f(x), f_(pa)(x))

(38.0 + -3.8 𝐞₁ + -6.35 𝐞₂ + -9.25 𝐞₃ + -3.82 𝐞₁₂ + 2.62 𝐞₁₃ + -1.57 𝐞₂₃ + -4.43 𝐞₁₂₃,
 38.0 + -3.8 𝐞₁ + -6.35 𝐞₂ + -9.25 𝐞₃ + -3.82 𝐞₁₂ + 2.62 𝐞₁₃ + -1.57 𝐞₂₃ + -4.43 𝐞₁₂₃)

In [10]:
#5.12c
# without explicit define the inverse of f, we push forward the derivative
def forward_derivative_gen(x, f, alg, Ix, h=1e-6):
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
    for v, r in frame_gen(x, alg, f_(Ix)):
        yield (r, lambda F: differential(F, f(x), v, h))

fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix)
sum((pa(x)|fa_(r)(x)) * o(G) for r, o in forward_derivative_gen(x, f, alg, Ix))

38.0 + -3.8 𝐞₁ + -6.35 𝐞₂ + -9.25 𝐞₃ + -3.82 𝐞₁₂ + 2.62 𝐞₁₃ + -1.57 𝐞₂₃ + -4.43 𝐞₁₂₃

In [11]:
#5.13
(
    derivative(F, x, alg, Ix=Ix), 
    sum(fa_(r)(x) * o(G) for r, o in forward_derivative_gen(x, f, alg, Ix))
    )

(10.0 + 12.7 𝐞₁ + -31.5 𝐞₂ + 3.7 𝐞₃ + -6.63 𝐞₁₂ + -9.54 𝐞₁₃ + 7.99 𝐞₂₃ + 4.14 𝐞₁₂₃,
 10.0 + 12.6 𝐞₁ + -31.5 𝐞₂ + 3.7 𝐞₃ + -6.63 𝐞₁₂ + -9.54 𝐞₁₃ + 7.99 𝐞₂₃ + 4.14 𝐞₁₂₃)

In [12]:
def outermorphism_(f, A, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        wf = lambda vectors: wedge([f(v)(x) for v in vectors])
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
        return outer
    return f_outer

In [13]:
#5.14
f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix)
(
    sum(r * o(G) for r, o in forward_derivative_gen(x, f, alg, Ix)),
    sum(inv(f_(nIx)(x)) * f__(nI*r)(x) * o(F) for r, o in derivative_gen(x, alg, Ix))
    )

(3.63 + -1.68 𝐞₁ + -3.14 𝐞₂ + -4.61 𝐞₃ + 0.326 𝐞₁₂ + -0.3 𝐞₁₃ + 0.821 𝐞₂₃ + 1.5 𝐞₁₂₃,
 3.63 + -1.68 𝐞₁ + -3.14 𝐞₂ + -4.61 𝐞₃ + 0.326 𝐞₁₂ + -0.3 𝐞₁₃ + 0.821 𝐞₂₃ + 1.5 𝐞₁₂₃)

In [67]:
#5.15a Without the inverse of f
# Pushing forward a differential is an extra linear transformation
b = random_r_blade(1, alg)
pb = P(b, I)
pb1 = f__(pb)(x)
differential(f__(PA(x)), x, pb, h=1e-2), sum((pb1 | (inv(f_(nIx)(x)) * f__(nI*r)(x))) * o(f__(PA(x))) for r, o in derivative_gen(x, alg, Ix, h=1e-2))

(2.25 𝐞₁ + 0.633 𝐞₂ + 5.33 𝐞₃ + -0.0144 𝐞₁₂ + -0.0726 𝐞₁₃ + 0.078 𝐞₂₃ + 2.04e-16 𝐞₁₂₃,
 5.62e-18 + 2.24 𝐞₁ + 0.655 𝐞₂ + 5.28 𝐞₃ + -0.0147 𝐞₁₂ + -0.0717 𝐞₁₃ + 0.0771 𝐞₂₃ + -2.11e-15 𝐞₁₂₃)

In [68]:
#5.15b
differential(fa_(PA(x)), x, pb, h=1e-2), sum((pb1 | (inv(f_(nIx)(x)) * f__(nI*r)(x))) * o(fa_(PA(x))) for r, o in derivative_gen(x, alg, Ix, h=1e-2))

(-0.466 𝐞₁ + 0.326 𝐞₂ + -0.0098 𝐞₃ + -0.0142 𝐞₁₂ + 0.0022 𝐞₁₃ + -0.00235 𝐞₂₃,
 -2.1e-19 + -0.469 𝐞₁ + 0.349 𝐞₂ + -0.0104 𝐞₃ + -0.0145 𝐞₁₂ + 0.00221 𝐞₁₃ + -0.00234 𝐞₂₃ + 3.07e-17 𝐞₁₂₃)

In [92]:
#5.16
(
    differential(f__(C^D), x, pb, h=1e-3), 
    (differential(f__(C), x, pb, h=1e-3)^f__(D)(x)) + (f__(C)(x)^differential(f__(D), x, pb, h=1e-3)),
    differential(fa_(C^D), x, pb, h=1e-3), 
    (differential(fa_(C), x, pb, h=1e-3)^fa_(D)(x)) + (fa_(C)(x)^differential(fa_(D), x, pb, h=1e-3)),
    )

(-0.143 𝐞₁ + -0.0663 𝐞₂ + -4.39 𝐞₃ + -3.02 𝐞₁₂ + -17.9 𝐞₁₃ + 13.9 𝐞₂₃ + 4.62e-14 𝐞₁₂₃,
 -0.143 𝐞₁ + -0.0663 𝐞₂ + -4.39 𝐞₃ + -3.02 𝐞₁₂ + -17.9 𝐞₁₃ + 13.9 𝐞₂₃ + -0.000897 𝐞₁₂₃,
 0.431 𝐞₁ + -5.15 𝐞₂ + 0.287 𝐞₃ + -13.7 𝐞₁₂ + 1.52 𝐞₁₃ + -1.4 𝐞₂₃,
 0.431 𝐞₁ + -5.15 𝐞₂ + 0.287 𝐞₃ + -13.7 𝐞₁₂ + 1.51 𝐞₁₃ + -1.4 𝐞₂₃ + -0.000672 𝐞₁₂₃)

In [123]:
#5.17a
Pb_ = lambda A: lambda x: differential(lambda a: P(A(x), Ix(a)), x, pb, h=1e-2)
Pb_1 = lambda A: lambda x: differential(lambda a: P(A(x), f_(Ix)(a)), x, pb, h=1e-2)
fb = lambda C: differential(f__(C), x, pb, h=1e-2)
C_ = lambda x: C
(
    fb(C),
    Pb_1(f__(C))(x)+ P(fb(C), I1),
    fb(P(C, I)) + f_(Pb_(C_))(x)
    )

(0.0504 𝐞₁ + 0.07 𝐞₂ + -3.64 𝐞₃ + -1.73 𝐞₁₂ + -9.01 𝐞₁₃ + 9.84 𝐞₂₃ + 2.02e-15 𝐞₁₂₃,
 5.55e-15 + 0.0505 𝐞₁ + 0.0696 𝐞₂ + -3.64 𝐞₃ + -1.73 𝐞₁₂ + -9.01 𝐞₁₃ + 9.84 𝐞₂₃,
 -1.67e-14 + 0.0503 𝐞₁ + 0.07 𝐞₂ + -3.64 𝐞₃ + -1.73 𝐞₁₂ + -9.01 𝐞₁₃ + 9.84 𝐞₂₃ + 2.03e-15 𝐞₁₂₃)

In [136]:
#5.17b
fb = lambda C: differential(fa_(C), x, pb, h=1e-2)

(
    fb(C),
    Pb_(fa_(C))(x)+ P(fb(C), I),
    fb(P(C, I1)) + fa_(Pb_1(C_)(x))(x)
    )

(0.556 𝐞₁ + -2.92 𝐞₂ + 0.207 𝐞₃ + -6.19 𝐞₁₂ + 0.762 𝐞₁₃ + -0.743 𝐞₂₃,
 -1.67e-14 + 0.556 𝐞₁ + -2.92 𝐞₂ + 0.207 𝐞₃ + -6.19 𝐞₁₂ + 0.762 𝐞₁₃ + -0.743 𝐞₂₃,
 5.55e-15 + 0.556 𝐞₁ + -2.92 𝐞₂ + 0.207 𝐞₃ + -6.19 𝐞₁₂ + 0.762 𝐞₁₃ + -0.743 𝐞₂₃)

In [140]:
#5.18
differential(lambda y: differential(f, y, a(x)), x, b), differential(lambda y: differential(f, y, b), x, a(x))

(9.15 𝐞₁ + -1.4 𝐞₂ + 20.7 𝐞₃, 9.15 𝐞₁ + -1.4 𝐞₂ + 20.7 𝐞₃)

In [159]:
#5.19
tangents = blade_split(I, alg)
frame = multi_frame(tangents)
r_frame = reci_frame(tangents)
# pb_frame = [differential(lambda x: P(v, Ix(x)), x, b) for v in frame]
A1 = f_(PA)(x)
fb = lambda C: differential(fa_(C), x, pb, h=1e-2)
fb_A1 = lambda C: differential(f__(C), x, pb, h=1e-2).sp(A1)

(
    derivative(fb_A1, x, alg, frame=frame, r_frame=r_frame, h=1e-2) +
    Pb_(fa_(A1))(x),
    fb(A1)
    )

(5.11 𝐞₁ + 27.2 𝐞₂ + -0.805 𝐞₃ + -1.29 𝐞₁₂ + 0.107 𝐞₁₃ + -0.0823 𝐞₂₃ + 6.76e-17 𝐞₁₂₃,
 5.11 𝐞₁ + 27.2 𝐞₂ + -0.805 𝐞₃ + -1.29 𝐞₁₂ + 0.107 𝐞₁₃ + -0.0823 𝐞₂₃)

In [161]:
#5.20
(
    derivative(fb_A1, x, alg, frame=frame, r_frame=r_frame, h=1e-2),
    P(Pb_(fa_(A1))(x), I),
    P(fb(A1), I)
    )

(5.11 𝐞₁ + 27.2 𝐞₂ + -0.555 𝐞₃ + -1.29 𝐞₁₂ + 0.031 𝐞₁₃ + 0.0249 𝐞₂₃ + 6.76e-17 𝐞₁₂₃,
 3.63e-06 𝐞₁ + 3.52e-06 𝐞₂ + -1.46e-08 𝐞₃ + 5.18e-08 𝐞₁₂ + -1.24e-09 𝐞₁₃ + -9.97e-10 𝐞₂₃,
 5.11 𝐞₁ + 27.2 𝐞₂ + -0.555 𝐞₃ + -1.29 𝐞₁₂ + 0.031 𝐞₁₃ + 0.0249 𝐞₂₃)

In [162]:
#5.21
C.sp(fb(A1)), fb_A1(C)

(19.2, 19.2)

In [198]:
#5.22 quadriple differential, so the truncations...
f_vec = lambda a: lambda x: differential(f, x, a, h=1e-3)
fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
fb = lambda b: differential(fa_(A1), x, b, h=1e-3)
P(curl(fb, x, alg, Ix=Ix, h=1e-2), I)

-8.88e-19 𝐞₂ + 0.000509 𝐞₁₂ + -1.22e-05 𝐞₁₃ + -9.8e-06 𝐞₂₃

In [199]:
#5.23
f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
d(lambda x, A: f__(A)(x), x, PA(x), alg, Ix, h=1e-2)

6.36e-07 𝐞₁ + -6.6e-07 𝐞₂ + 3.23e-07 𝐞₃ + 1.46e-18 𝐞₁₂ + 2.28e-18 𝐞₁₃ + 2.51e-19 𝐞₂₃ + 6.28e-17 𝐞₁₂₃

In [ ]:
#